An exploratory notebook looking at the results of a Naive Bayes classifier prior to looking at the leaderboard or other resources.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import metrics

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
comp_path = 'kaggle/input/lish-moa/'

In [ ]:
tf_df = pd.read_csv(comp_path+'train_features.csv',index_col='sig_id')

In [ ]:
pca = PCA(n_components=50)
pca.fit(tf_df.loc[:,'g-0':'c-99'])
pca.explained_variance_ratio_

Large drops after 2, 6, 8, 12 components.

In [ ]:
reduced_comps = pca.transform(tf_df.loc[:,'g-0':'c-99'])

In [ ]:
reduced_comps = reduced_comps[:,:6]

In [ ]:
tts_df = pd.read_csv(comp_path+'train_targets_scored.csv',index_col='sig_id')

In [ ]:
tts_df.columns

In [ ]:
nb_model = MultiOutputClassifier(GaussianNB())
nb_model.fit(reduced_comps,tts_df)
tts_pred = nb_model.predict(reduced_comps)
metrics.log_loss(tts_df,tts_pred)

It happens that we can predict column 1 quite well, but on average, worse than 21%. Looking at the results, I think that for column 1 the model simply decided to predict 0 for all runs because only 17 out of 2x,xxx rows had a 1.

Logistic regression (saga, L1 regularization, 5000 `max_iter`) was "better", *i.e.* 42% accuracy, although that took over an hour to squeeze out on my antiquated hardware.

I also checked and it turns out that throwing 8 PCA components at NB made the model *worse*, and all 50 PCA components made it a *lot* worse at predicting column 1.

No wonder they want people pulling out deep learning to attack this.

Would be good to cycle through the whole list and see which columns perform how.

Oh, and interesting to see if any of the PCA components got zero coefficients under L1 regularization... for each target... that's a lot to check.

Can take this, spit it out and upload it to Kaggle as a baseline along with the logistic regression results.

In [ ]:
(pd.DataFrame(tts_pred,columns=tts_df.columns,index=tts_df.index)).to_csv(path_or_buf=
            'submission.csv')